In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

import numpy as np

In [8]:
def get_linear_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
            
        for qubit_index in range(num_of_qubits - 1):
            quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[qubit_index + 1]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

In [9]:
def get_full_entangelment_ansatz(num_of_qubits, thetas, input_state, circuit_depth=3):
    quantum_register = QuantumRegister(num_of_qubits, name="qubit")
    quantum_circuit = QuantumCircuit(quantum_register)
    quantum_circuit.initialize(input_state)

    for iteration in range(circuit_depth):
        for qubit_index in range(num_of_qubits):
            RY_theta_index = iteration*2*num_of_qubits + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])
        for qubit_index in range(num_of_qubits - 1):
            for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                quantum_circuit.append(CXGate(), [quantum_register[qubit_index], quantum_register[target_qubit_index]])

    for qubit_index in range(num_of_qubits):
            RY_theta_index = 2*num_of_qubits*circuit_depth + qubit_index
            RZ_theta_index = RY_theta_index + num_of_qubits
            
            quantum_circuit.append(RYGate(thetas[RY_theta_index]), [quantum_register[qubit_index]])
            quantum_circuit.append(RZGate(thetas[RZ_theta_index]), [quantum_register[qubit_index]])

    return quantum_circuit

## Linear entangelment ansatz

In [10]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_linear_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐ ┌────────────┐ ┌────────────┐      ┌────────────┐»
qubit_0: ┤0               ├─┤ Ry(4.9434) ├─┤ Rz(6.2828) ├───■──┤ Ry(5.0149) ├»
         │                │┌┴────────────┤ ├────────────┤ ┌─┴─┐└────────────┘»
qubit_1: ┤1               ├┤ Ry(0.17499) ├─┤ Rz(4.2696) ├─┤ X ├──────■───────»
         │  Initialize(3) │└┬───────────┬┘ ├────────────┤ └───┘    ┌─┴─┐     »
qubit_2: ┤2               ├─┤ Ry(3.844) ├──┤ Rz(1.4013) ├──────────┤ X ├─────»
         │                │ ├───────────┴┐┌┴────────────┴┐         └───┘     »
qubit_3: ┤3               ├─┤ Ry(2.0786) ├┤ Rz(0.046831) ├───────────────────»
         └────────────────┘ └────────────┘└──────────────┘                   »
«         ┌────────────┐                             ┌────────────┐»
«qubit_0: ┤ Rz(4.4632) ├─────────────────────■───────┤ Ry(4.1847) ├»
«         ├────────────┤ ┌────────────┐    ┌─┴─┐     └────────────┘»
«qubit_1: ┤ Ry(1.3892) ├─┤ Rz(2.5153) ├────┤ X ├───────────■───────»
«         └──

## Full entangelment ansatz

In [11]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=32)
initial_eigenvector = np.identity(16)[0]
print(get_full_entangelment_ansatz(4, initial_thetas, 3))

         ┌────────────────┐┌────────────┐┌─────────────┐               »
qubit_0: ┤0               ├┤ Ry(4.6619) ├┤ Rz(0.41515) ├──■────■────■──»
         │                │├────────────┤└┬────────────┤┌─┴─┐  │    │  »
qubit_1: ┤1               ├┤ Ry(4.8225) ├─┤ Rz(5.0193) ├┤ X ├──┼────┼──»
         │  Initialize(3) │├────────────┤ ├────────────┤└───┘┌─┴─┐  │  »
qubit_2: ┤2               ├┤ Ry(2.9944) ├─┤ Rz(3.6326) ├─────┤ X ├──┼──»
         │                │├────────────┤ ├───────────┬┘     └───┘┌─┴─┐»
qubit_3: ┤3               ├┤ Ry(3.5793) ├─┤ Rz(1.981) ├───────────┤ X ├»
         └────────────────┘└────────────┘ └───────────┘           └───┘»
«         ┌───────────┐┌────────────┐                                          »
«qubit_0: ┤ Ry(4.902) ├┤ Rz(4.5362) ├──────────────────────────────────■───────»
«         └───────────┘└────────────┘┌────────────┐┌────────────┐    ┌─┴─┐     »
«qubit_1: ──────■────────────■───────┤ Ry(5.3632) ├┤ Rz(5.8175) ├────┤ X ├─────»
«             ┌─┴─┐